# Initial Hamming Weight experiment

With brute-force hyper parameter optimization of the MLP.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.python.keras.optimizer_v2.adam import Adam
from tqdm.keras import TqdmCallback

from src.data.ascad import AscadRandomKey
from src.dlla.hw import NUM_CLASSES, fetch_traces, plot_predictions, dlla_hw, dlla_p_gradient, encode, split_traces
from src.dlla.wegener import make_mlp_wegener, wegener_p_gradient
from src.tools.cache import cache_np
from src.tools.lists import random_divide, randomize
from src.tools.plotter import init_plots, plot_p_gradient, PALETTE_GRADIENT, store_sns
from src.trace_set.database import Database
from src.trace_set.pollution import Pollution, PollutionType
from src.trace_set.set_hw import TraceSetHW
from src.trace_set.transform import reduce_fixed_fixed
from src.tvla.cri import tvla_cri, tvla_cri_p_gradient
from src.tvla.tvla import Tvla

init_plots()

In [5]:
DB = Database.ascad
TRACE_SET = TraceSetHW(DB)

In [4]:
def prepare_traces_dl(x, y, x_att, y_att):
    """
    Normalizes the traces, one-hot encodes the labels.
    Returns profiling traces, labels and attack traces, labels.
    """
    prof_mean, prof_std = x.mean(axis=0), x.std(axis=0)
    norm_x = (x - prof_mean) / prof_std
    norm_x_att = (x_att - prof_mean) / prof_std

    return norm_x, encode(y), norm_x_att, encode(y_att)

X, Y, X_ATT, Y_ATT = prepare_traces_dl(*TRACE_SET.profile(), *TRACE_SET.attack())

In [ ]:
TRACE_LENGTH = X.shape[1]

### Model creation

With model hyper-parameters to be optimized.

In [ ]:
def make_mlp(x, y, x_attack, y_attack, params):
    mdl = Sequential()
    mdl.add(Dense(100, activation=params['activation'], input_shape=(x.shape[1],)))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(NUM_CLASSES, activation='softmax'))

    mdl.compile(optimizer=params['optimizer'], loss=params['losses'], metrics=['accuracy'])

    out = mdl.fit(x, y, shuffle=True, validation_data=(x_attack, y_attack), batch_size=params['batch_size'],
                  epochs=params['epochs'], verbose=False, callbacks=[TqdmCallback(verbose=0)])

    return out, mdl

### Brute-force Hyper parameter optimization

Talos brute-force scan for optimal hyperparameters

In [ ]:
# import talos
#
# scan_obj = talos.Scan(x=x_prof,    y=y_prof,    x_val=x_att,    y_val=y_att,    model=make_mlp,    params=model_parameters,    experiment_name=".cache/talos", print_params=True)

### Fit model based on the optimal model found by Talos.

In [ ]:
best_model = make_mlp(X, Y, X_ATT, Y_ATT, {
    'activation':'relu',
    'optimizer': Adam(learning_rate=0.001),
    'losses': 'categorical_crossentropy',
    'batch_size': 150,
    'epochs': 5
})[1]

### HW Prediction

Predict the hamming weight by taking the weighted mean for the predicted probabilities for each class.

#### Example: Some prediction

Probabilities for each class:

In [ ]:
best_model.predict(X_ATT[:1])[0]

**Predicted hamming weight label**, calculated by taking the weighted mean using the predicted probabilities.

In [ ]:
np.sum(best_model.predict(X_ATT[:1])[0] * range(8 + 1))

In [ ]:
dlla_hw(best_model, X_ATT, Y_ATT)

### Plot prediction distribution for all traces.

In [ ]:
# store_sns(plot_predictions(best_model, X_ATT, Y_ATT), "dlla-9-predictions")

### Plot p-gradient.

In [ ]:
A_ATT, B_ATT = split_traces(X_ATT, Y_ATT, balance=True)

In [ ]:
MAX_LEN = round(len(A_ATT) / 2)
TVLA_AB = Tvla(A_ATT.shape[1])
TVLA_AB.add(A_ATT[:MAX_LEN], B_ATT)
TVLA_BB = Tvla(TRACE_LENGTH)
TVLA_BB.add(*random_divide(B_ATT))

In [ ]:
plot_p_gradient(dict([(f"$\\mu_{{{d}}}$", TVLA_AB.p_gradient(d)) for d in range(1, 4)]),
                "TVLA performance, first 3 statistical moment orders\nMasked AES with 1400 sample points", palette=PALETTE_GRADIENT)

In [ ]:
G = TVLA_AB.plot_p_values(2)
store_sns(G, "p-values-ascad")

In [ ]:
MAX_WEIGHTS = np.max(best_model.layers[0].get_weights()[0], axis=1)

sns.lineplot(data=MAX_WEIGHTS)
AX2 = plt.twinx()
G = sns.lineplot(data=TVLA_AB.min_p[2], ax=AX2, color=sns.color_palette()[1], alpha=.5)
G.set(yscale="log")
G.invert_yaxis()

In [ ]:
print("p-value for uncorrelated system producing these datasets")
pearsonr(MAX_WEIGHTS, TVLA_AB.min_p[2])[1]

In [ ]:
plot_p_gradient({
    "A vs. B": TVLA_AB.p_gradient(2),
    "FP check": TVLA_BB.p_gradient(2)
}, "TVLA min-$p$ performance validation ($t$-test for $\\mu_2$ with min-$p$)\nMasked AES with 1400 sample points",
    palette=PALETTE_GRADIENT,
    file_name="tvla-validation-mu2")

PG_LEN = len(TVLA_AB.p_gradient(2))

In [ ]:
CRI_AB = cache_np("cri_tvla_pg_ab", tvla_cri_p_gradient, TRACE_SET, 2, False, None, 1)
CRI_RANDOM = cache_np("cri_tvla_pg_random", tvla_cri_p_gradient, TRACE_SET, 2, True, None, 1)

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

NUM_TRACES = len(X)
PG_CRI_AB = np.interp(range(NUM_TRACES), moving_average(CRI_AB[0], 5), moving_average(CRI_AB[1], 5))
PG_CRI_RANDOM = np.interp(range(NUM_TRACES), moving_average(CRI_RANDOM[0], 5), moving_average(CRI_RANDOM[1], 5))

In [ ]:
# PG_CRI_RANDOM = np.interp(range(NUM_TRACES), CRI_RANDOM[0], CRI_RANDOM[1])

plot_p_gradient({
    "A vs. B": PG_CRI_AB,
    "Random": PG_CRI_RANDOM,
}, "TVLA performance validation ($t$-test for $\\mu_2$ with CRI min-$p$)\nMasked AES with 1400 sample points",
    palette=PALETTE_GRADIENT,
    file_name="tvla-cri-validation-mu2")

In [ ]:
PG_DLLA_9 = dlla_p_gradient(best_model, X_ATT, Y_ATT)["A vs. B"]
PG_DLLA_9_RANDOM = dlla_p_gradient(best_model, X_ATT, randomize(Y_ATT))["A vs. B"]

plot_p_gradient({
    "A vs. B": np.array(PG_DLLA_9),
    "FP check": np.array(PG_DLLA_9_RANDOM),
}, "DL-LA performance validation (9-class)\nMasked AES with 1400 sample points",
    palette=PALETTE_GRADIENT,
    file_name="dlla-9-validation"
)

In [ ]:
X_PROF_2, Y_PROF_2 = reduce_fixed_fixed(X, Y)
X_ATT_2, Y_ATT_2 = reduce_fixed_fixed(X_ATT, Y_ATT)
Y_ATT_2_RANDOM = randomize(Y_ATT_2)

In [ ]:
MODEL_WEGENER_AB = make_mlp_wegener(X_PROF_2, Y_PROF_2)

PG_DLLA_2_AB = wegener_p_gradient(MODEL_WEGENER_AB, X_ATT_2[:PG_LEN], Y_ATT_2[:PG_LEN])
PG_DLLA_2_RANDOM =  wegener_p_gradient(MODEL_WEGENER_AB, X_ATT_2[:PG_LEN], Y_ATT_2_RANDOM[:PG_LEN])

plot_p_gradient({
    "A vs. B": np.array(PG_DLLA_2_AB),
    "FP check": np.array(PG_DLLA_2_RANDOM),
}, "DL-LA performance validation (Wegener)\nMasked AES with 1400 sample points",
    palette=PALETTE_GRADIENT,
    file_name="dlla-2-validation"
)

In [ ]:
plot_p_gradient({
    "DL-LA (9-class)": np.array(PG_DLLA_9),
    "DL-LA (Wegener)": np.array(PG_DLLA_2_AB),
    "TVLA $\\mu_2$ (min-$p$)": TVLA_AB.p_gradient(2)
},"LA method performance\nmasked AES with 1400 sample points",
    file_name="ascad-default-all"
)

In [ ]:
PGS = {
    "DL-LA 9-class": np.repeat(np.array(PG_DLLA_9), 2)[:1000],
    "DL-LA Wegener": np.array(PG_DLLA_2_AB[:1000]),
}

DF = pd.DataFrame(PGS)
DF.to_csv("dlla-p-gradient.csv")
plot_p_gradient(PGS, "LA method performance\nmasked AES with 1400 sample points",
    file_name="ascad-default-dlla-1000"
)
